In [ ]:
!wget https://point-clouds-data.s3.us-west-2.amazonaws.com/0000000357.pcd

# Reflectance duplicated into 3 color channels for visualization, not real camera color all at 1st channel (Red)

In [ ]:
!wget https://point-clouds-data.s3.us-west-2.amazonaws.com/KITTI_PCD.zip && unzip KITTI_PCD.zip

In [ ]:
!mkdir output

In [ ]:
!pip install open3d

In [ ]:
!pip install kaleido==0.2.1

In [ ]:
import open3d as o3d
import numpy as np
import glob
import matplotlib.pyplot as plt
from open3d.visualization.draw_plotly import get_plotly_fig


In [ ]:
point_cloud_files=sorted(glob.glob("KITTI_PCD/*.pcd"))
idx=200
print(point_cloud_files[idx])
point_cloud=o3d.io.read_point_cloud(point_cloud_files[idx])

In [ ]:
points=np.asarray(point_cloud.points)
colors=np.asarray(point_cloud.colors)

In [ ]:
print(points)

In [ ]:
print(colors) # Values of reflectance

In [ ]:
import plotly.graph_objects as go

distances = np.linalg.norm(points, axis=1)

fig = go.Figure(data=[go.Scatter3d(
    x=points[:, 0],
    y=points[:, 1],
    z=points[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=distances,  # use distances for color
        colorscale='Viridis',  # choose a colorscale
        colorbar=dict(title="Distance from Origin"),  # add a colorbar title
        opacity=0.8
    )
)])
fig.update_scenes(aspectmode='data')

fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=points[:, 0],
    y=points[:, 1],
    z=points[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=distances,  # use distances for color
        colorscale='Inferno',  # choose a colorscale
        colorbar=dict(title="Distance from Origin", bgcolor="white"),  # add a colorbar title
        opacity=0.8
    )
)])

fig.update_layout(
    scene=dict(
        xaxis=dict(showbackground=False, showline=False, zeroline=False, showgrid=False, showticklabels=False, title=''),
        yaxis=dict(showbackground=False, showline=False, zeroline=False, showgrid=False, showticklabels=False, title=''),
        zaxis=dict(showbackground=False, showline=False, zeroline=False, showgrid=False, showticklabels=False, title=''),
        aspectmode='data',
        camera=dict(
            up=dict(x=-0.2, y=0, z=1),
            center=dict(x=0.2, y=0, z=0.2),
            eye=dict(x=-0.5, y=0, z=0.2))
    ),
    plot_bgcolor='black',
    paper_bgcolor='black',
    scene_dragmode='orbit'
)
fig.show()

In [ ]:
from open3d.visualization.draw_plotly import get_plotly_fig             # Simply visualizing points using reflectance and dark mode

def vis_pcd(point_cloud, save="False", show=True):
    fig = get_plotly_fig(point_cloud, width = 800, height = 533, mesh_show_wireframe =False,
                            point_sample_factor = 1, front = (1,1,1), lookat =(1,1,1), up=(1,1,1), zoom=1.0)
    #fig.update_scenes(aspectmode='data')
    fig.update_layout(
    scene=dict(
        #xaxis=dict(showbackground=False, showline=False, zeroline=False, showgrid=False, showticklabels=False, title=''),
        xaxis=dict(visible=False,range=[-70,70]),
        yaxis=dict(visible=False,range=[-40,40]),
        zaxis=dict(visible=False,range = [-5,1]),
        aspectmode='manual', aspectratio= dict(x=2, y=1, z=0.1),
        camera=dict(
            #up=dict(x=-0.2, y=0, z=0.3),
            up = dict(x=0.15, y =0, z=1),
            center=dict(x=0, y=0, z=0.1),
            #eye=dict(x=-0.5, y=0, z=0.2)
            eye = dict(x = -0.3, y=0, z=0.2)
        )
    ),
    plot_bgcolor='black',
    paper_bgcolor='black',
    scene_dragmode='orbit'
)
    if show == True:
        fig.show()

    return fig


point_cloud = o3d.io.read_point_cloud(point_cloud_files[200])
fig = vis_pcd([point_cloud], save="1")


In [ ]:
def reflectivity_threshold(pcd, thresh=0.5):
    colors = np.asarray(pcd.colors)
    reflectivities = colors[:, 0]
    # Get the point coordinates
    points = np.asarray(pcd.points)
    # Create a mask of points that have reflectivity above the threshold
    mask = reflectivities > thresh

    # Filter points and reflectivities using the mask
    filtered_points = points[mask]
    filtered_reflectivities = reflectivities[mask]

    # Create a new point cloud with the filtered points
    filtered_point_cloud = o3d.geometry.PointCloud()
    filtered_point_cloud.points = o3d.utility.Vector3dVector(filtered_points)
    return filtered_point_cloud

#point_cloud = o3d.io.read_point_cloud(point_cloud_files[idx])
filtered_point_cloud = reflectivity_threshold(point_cloud, thresh=0.45)

In [ ]:
fig = vis_pcd([point_cloud.paint_uniform_color((0.2,0.2,0.2)),filtered_point_cloud])

In [ ]:
def roi_filter(pcd, roi_min=(0,-3,-2), roi_max=(20,3,0)):
    points = np.asarray(pcd.points)

    mask_roi = np.logical_and.reduce((
        points[:, 0] >= roi_min[0],
        points[:, 0] <= roi_max[0],
        points[:, 1] >= roi_min[1],
        points[:, 1] <= roi_max[1],
        points[:, 2] >= roi_min[2],
        points[:, 2] <= roi_max[2]
    ))

    roi_points = points[mask_roi]

    # Create a new point cloud with the filtered points
    roi_pcd = o3d.geometry.PointCloud()
    roi_pcd.points = o3d.utility.Vector3dVector(roi_points)
    return roi_pcd

roi_pcd = roi_filter(filtered_point_cloud)
fig = vis_pcd([point_cloud.paint_uniform_color((0.2,0.2,0.2)),roi_pcd])

In [ ]:
def lane_line_pipeline(pcd):
    filtered_point_cloud = reflectivity_threshold(pcd)
    roi_pcd = roi_filter(filtered_point_cloud)
    return roi_pcd, filtered_point_cloud, pcd

point_cloud = o3d.io.read_point_cloud(point_cloud_files[201])
roi_pcd, filtered_point_cloud, pcd = lane_line_pipeline(point_cloud)

fig = vis_pcd([pcd.paint_uniform_color((0.2,0.2,0.2)),roi_pcd], show=True)

In [ ]:
pcd = o3d.io.read_point_cloud(point_cloud_files[200]) # 200 has some objects
print(len(pcd.points))

In [ ]:
downpcd = pcd.voxel_down_sample(voxel_size=0.1)
print(len(downpcd.points))

In [ ]:
vis_pcd([downpcd])

In [ ]:
def ransac(pcd, distance_threshold=0.33, ransac_n=3, num_iterations=100):
    plane_model, inliers = downpcd.segment_plane(distance_threshold=distance_threshold, ransac_n=ransac_n, num_iterations=num_iterations)
    inlier_cloud = downpcd.select_by_index(inliers)
    outlier_cloud = downpcd.select_by_index(inliers, invert=True)
    outlier_cloud.paint_uniform_color([0.5, 0.75, 0.25])
    inlier_cloud.paint_uniform_color([0.25, 0.5, 0.75])
    return inlier_cloud, outlier_cloud

inlier_cloud, outlier_cloud = ransac(downpcd, distance_threshold=0.3, ransac_n=3, num_iterations=200)
fig = vis_pcd([inlier_cloud, outlier_cloud])

In [ ]:
def dbscan(outlier_cloud, eps = 1, min_points=15):
    # You can also use an ROI Filter here.
    outlier_cloud = roi_filter(outlier_cloud, roi_min=(-20, -8, -2), roi_max=(20,8,0))
    labels = np.array(outlier_cloud.cluster_dbscan(eps=eps, min_points=min_points))
    max_label = labels.max()
    colors = plt.get_cmap("inferno_r")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    outlier_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
    return outlier_cloud, labels

roi_outliers, labels = dbscan(outlier_cloud, eps=1, min_points=15)
fig = vis_pcd([inlier_cloud, roi_outliers])

In [ ]:
print(labels)
print(len(labels))

In [ ]:
def get_bounding_boxes(labels, outlier_cloud):
    # Extract points for each cluster
    clusters = {}
    for i, label in enumerate(labels):
        if label >= 0:
            if label not in clusters:
                clusters[label] = []
            clusters[label].append(outlier_cloud.points[i])

    clusters = {label: points for label, points in clusters.items() if len(points) <= 900}

    # Create AABB for each cluster
    aabb_boxes = []
    for points in clusters.values():
        cluster_cloud = o3d.geometry.PointCloud()
        cluster_cloud.points = o3d.utility.Vector3dVector(points)
        aabb = cluster_cloud.get_axis_aligned_bounding_box()
        aabb_boxes.append(aabb)
    return aabb_boxes

aabb_boxes = get_bounding_boxes(labels, roi_outliers)
print(len(aabb_boxes))

In [ ]:
def get_trace(obb_boxes, fig):
    width = 1.0
    height = 2.0
    depth = 3.0
    for obb in obb_boxes:
        # Get the eight corner points of the OBB
        corners = np.asarray(obb.get_box_points())

        # Extract x, y, and z coordinates of the corners
        x = corners[:, 0]
        y = corners[:, 1]
        z = corners[:, 2]
        # Create a Mesh3d trace for the oriented bounding box with opacity
        obb_trace = go.Mesh3d(
            x=x,
            y=y,
            z=z,
            i=[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 2, 6, 4, 1, 3, 7, 5],
            j=[1, 2, 3, 0, 5, 6, 7, 4, 2, 3, 7, 6, 1, 0, 4, 5, 6, 7, 3, 2, 0, 1, 5, 4],
            k=[2, 3, 0, 1, 6, 7, 4, 5, 3, 7, 6, 2, 0, 4, 5, 1, 7, 6, 2, 4, 1, 5, 0, 3],
            color='blue',
            opacity=0.3
        )

        # Add the Mesh3d trace to the figure
        fig.add_trace(obb_trace)
    return fig

In [ ]:
def pipeline(pcd):
    inlier_cloud, outlier_cloud = ransac(pcd, distance_threshold=0.3, ransac_n=3, num_iterations=200)
    roi_outliers, labels = dbscan(outlier_cloud)
    aabb_boxes = get_bounding_boxes(labels, roi_outliers)
    fig = vis_pcd([roi_outliers, inlier_cloud], show=False)
    fig = get_trace(aabb_boxes, fig)
    fig.show()

In [ ]:
def seg_pipeline(pcd, idx):
    downpcd = pcd.voxel_down_sample(voxel_size=0.2)
    inlier_cloud, outlier_cloud = ransac(downpcd, distance_threshold=0.3, ransac_n=3, num_iterations=200)
    roi_outliers, labels = dbscan(outlier_cloud)
    aabb_boxes = get_bounding_boxes(labels, roi_outliers)
    fig = vis_pcd([downpcd, roi_outliers, inlier_cloud], show=False)
    fig = get_trace(aabb_boxes, fig)
    fig.write_image("output/"+str(idx)+"_processed_obj.jpg", scale=3)
    fig = go.Figure()

In [ ]:
pipeline(pcd)

In [ ]:
import cv2
from tqdm import tqdm

output_handle = cv2.VideoWriter("objects_output.avi", cv2.VideoWriter_fourcc(*'XVID'), 10, (2400, 1599))

start_index = 250
stop_index = 400
pbar = tqdm(total = (stop_index - start_index), position=0, leave=True)
all_files = [o3d.io.read_point_cloud(point_cloud_files[i]) for i in range(start_index, stop_index)]

for i in range(len(all_files)):
    seg_pipeline(all_files[i], str(start_index+i))
    output_handle.write(cv2.imread("output/"+str(start_index+i)+"_processed_obj.jpg"))
    pbar.update(1)

output_handle.release()